In [ ]:
# default_exp data_structures

# Data Structures for Personnel Scheduling


At the moment, these are mostly structures for mulit-activity shift scheduling.

Some basic assumptions here:
- an activy may have different types. If the actitivy type is an integer, it is a work activity, other actitity types are characterized using strings such as `'break'`
- time is simply modeled using integer periods staring with 0




In [ ]:
#hide
from nbdev.showdoc import *

In [ ]:
#export

import importlib
from collections import namedtuple

from box import Box
import numpy as np
import pandas as pd
from dataclasses import dataclass, field



## Helper Functions

In [ ]:
#export

def is_work(activity_type):
    return activity_type is not None and (type(activity_type) != str or activity_type.isnumeric())



In [ ]:
#export

def get_end_period_block(start_period_block,length_block):
    return (start_period_block + length_block-1)

def get_next_period(p):
    return p+1
    

## Data Structures for shift schedules

The following data structures are used to represent activity blocks, work blocks and shifts; they are hierarchically structured as follows:
- an acitivty block consists of an activity type and a period of time in which this activity is assigned in a shift
- a work block is a block of consecutive work activities 
- a shift contains all activity blocks and work and break blocks constituting a full shift
- a shift schedule is a set of shifts that, in a typcial (multiactivity) shift scheduling problem, constitutes a solution.

### Blocks


In [ ]:
#export

@dataclass(eq=True, frozen=True)
class Block:
    start_period: int
    end_period: int
        
    def get_start_period(self):
        return self.start_period
    
    def get_end_period(self):
        return self.end_period
    
    def get_number_of_periods(self):
        return self.end_period - self.start_period + 1
    
        
        

NameError: name 'dataclass' is not defined

### Activity Blocks

In [ ]:
#export

@dataclass(eq=True, frozen=True)
class ActivityBlock(Block):
    activity_type: str
    

    def is_work(self):
        return is_work(self.activity_type)
    
    def get_number_of_work_periods(self):
        if self.is_work():
            return self.get_number_of_periods()
        return 0
    
    def get_number_of_break_periods(self):
        if not self.is_work():
            return self.get_number_of_periods()
        return 0
    
    def covers_demand(self, p, act):
        if self.activity_type != act:
            return False
    
        if (self.start_period <= self.end_period):
            return p >= self.start_period and p <= self.end_period
        else:
            return p >= self.start_period or p <= self.end_period
        
    def get_list_representation(self):
        activity = ''
        if self.is_work():
            activity = self.activity_type
            
        return [activity for i in range(self.end_period - self.start_period + 1)]
    
    def get_block(self):
        return Block(self.start_period, self.end_period)
    

In [ ]:
# maybe we can wrap this in tests       
activity_block = ActivityBlock(0,2,'1')

activity_block.is_work()

activity_block.get_number_of_work_periods()

block = activity_block.get_block()



In [ ]:
same_activity_block = ActivityBlock(0,2,'1')

activity_block == same_activity_block

dict_activity_block = {}
dict_activity_block[activity_block] = 1
dict_activity_block[same_activity_block] = 2

dict_activity_block[activity_block]

2

In [ ]:
#export

def create_activity_blocks_from_list_or_array(activity_list):

    last_activity_type = activity_list[0]
        
    last_activity_start_index = 0
    
    index = 0    
    activity_blocks = []
    # create activity blocks:
    
    for activity_type in activity_list[1:]:
        index +=1
        
        if activity_type != last_activity_type: 

            # do not use first activity block if it is non-work
            if is_work(last_activity_type) or len(activity_blocks) > 0:
                activity_blocks.append(ActivityBlock(last_activity_start_index, index-1, last_activity_type))

            last_activity_type = activity_type
            last_activity_start_index = index            
        

    #if the last block is work, create the last block    
    if is_work(last_activity_type):
        activity_blocks.append(ActivityBlock(last_activity_start_index, index, last_activity_type))
        
    return activity_blocks
    

### Work Blocks


In [ ]:
#export

@dataclass
class WorkBlock:
    activity_blocks: list = field(default_factory=list)
        
    def get_start_period(self):
        return self.activity_blocks[0].get_start_period()
    
    def get_end_period(self):
        return self.activity_blocks[-1].get_end_period()
    
    def is_work(self):
        return True
    
    def get_number_of_periods(self):
        return self.get_end_period() - self.get_start_period() + 1
    
    def get_number_of_work_periods(self):
        return self.get_end_period() - self.get_start_period() + 1
        
    
    def get_number_of_break_periods(self):
        return 0
    
    def covers_demand(self, p, act):
        for activity_block in self.activity_blocks:
            if activity_block.covers_demand(p, act):
                return True            
        return False
    
    def get_list_representation(self):
        list_rep = []
        for block in activity_blocks:
            list_rep += block.get_list_representation()            
        return list_rep

In [ ]:
    
activity_blocks = [ActivityBlock(0,2,1), ActivityBlock(3,4,2)]
    
work_block = WorkBlock(activity_blocks)

print(work_block.get_number_of_work_periods())

work_block.get_list_representation()

5


[1, 1, 1, 2, 2]

In [ ]:
#export

def create_work_blocks_from_activity_blocks(activity_blocks):      
    blocks = []
    work_activity_blocks = []
    # create activity blocks:
    for activity_block in activity_blocks:
        
        if not activity_block.is_work():
            
            if len(work_activity_blocks) > 0:
                blocks.append(WorkBlock(work_activity_blocks))
                
            blocks.append(activity_block)
            
            work_activity_blocks = []
            
        else:   
            work_activity_blocks.append(activity_block)
      
    if len(work_activity_blocks) > 0:
        blocks.append(WorkBlock(work_activity_blocks))
        
    return blocks
    

We should be able to read also if there is not "break, but empty space"

In [ ]:
activity_blocks = create_activity_blocks_from_list_or_array([1, 1, 1, 'break', 2, 2])

create_work_blocks_from_activity_blocks(activity_blocks)

[WorkBlock(activity_blocks=[ActivityBlock(start_period=0, end_period=2, activity_type=1)]),
 ActivityBlock(start_period=3, end_period=3, activity_type='break'),
 WorkBlock(activity_blocks=[ActivityBlock(start_period=4, end_period=5, activity_type=2)])]

### Shifts


In [ ]:
#export

@dataclass
class Shift:
    activity_blocks : list = field(default_factory=list)
    work_and_break_blocks: list = field(default_factory=list)

    
    @classmethod
    def from_list_or_array(cls,activity_list):
        activity_blocks = create_activity_blocks_from_list_or_array(activity_list)
        work_and_break_blocks = create_work_blocks_from_activity_blocks(activity_blocks)        
        return cls(activity_blocks, work_and_break_blocks)
    
    @classmethod
    def from_activity_blocks(cls,activity_blocks):
        work_and_break_blocks = create_work_blocks_from_activity_blocks(activity_blocks)        
        return cls(activity_blocks, work_and_break_blocks)
    
    def get_start_period(self):
        return self.work_and_break_blocks[0].get_start_period()
    
    def get_end_period(self):
        return self.work_and_break_blocks[-1].get_end_period()
    
    def get_number_of_periods(self):
        self.get_end_period() - self.get_start_period() + 1
    
    def get_number_of_work_periods(self):        
        number_of_work_periods = 0        
        for block in self.work_and_break_blocks:
            number_of_work_periods += block.get_number_of_work_periods()          
        return number_of_work_periods
               
    def get_number_of_break_periods(self):
        number_of_break_periods = 0      
        for block in self.work_and_break_blocks:
            number_of_break_periods += block.get_number_of_break_periods()
        return number_of_break_periods
    
    def covers_demand(self, p, act):
        for block in self.work_and_break_blocks:
            if block.covers_demand(p, act):
                return True            
        return False
 
    def get_number_of_work_blocks(self):
        number_of_blocks = 0
        for block in work_and_break_blocks:
            if block.is_work():
                number_of_blocks+=1
        return number_of_blocks       
    
    def get_number_of_break_blocks(self):
        number_of_blocks = 0
        for block in work_and_break_blocks:
            if not block.is_work():
                number_of_blocks+=1
        return number_of_blocks       
    
    def get_list_representation(self):
        list_rep = []
        for block in self.activity_blocks:
                list_rep += block.get_list_representation()
        return list_rep
        
    
    def get_array_representation(self,number_of_periods):
        full_list = []
        
        array_representation = np.full(number_of_periods,'')
        
        i = self.get_start_period()
        
        for activity in self.get_list_representation():
            array_representation[i] = activity
            i +=1
            
        return array_representation

### ShiftSchedule

In [ ]:
#export

def turn_string_to_activity_number(activity):
    if activity == '':
        return activity
    else:
        return int(activity)

@dataclass
class ShiftSchedule:
    shifts : list = field(default_factory=list)    
        
    def get_demand_coverage(self, p, act):
        coverage = 0
        for shift in self.shifts:
            if shift.covers_demand(p,act):
                coverage +=1                
        return coverage
    
    
    def get_number_of_periods(self):
        max_number = 0
        for shift in self.shifts:
            max_number = max(max_number, shift.activity_blocks[-1].get_end_period()+1)
        return max_number
    
    def get_number_of_shifts(self):
        return len(self.shifts)
    

        
    @classmethod
    def from_list_or_array(cls, activity_lists):
        shifts = []
        for activity_list in activity_lists:
            shifts.append(Shift.from_list_or_array(activity_list))
        return cls(shifts)
    
    @classmethod
    def from_file(cls, filename):
        f = open(filename)    
        lines = f.readlines()
        shifts = []
        for  line in lines:
            activity_list = [turn_string_to_activity_number(activity) for activity in line.rstrip().split(';')]
            shifts.append(Shift.from_list_or_array(activity_list))
        return cls(shifts)
    
    def write_to_csv(self, filename):
        shift_array = [shift.get_array_representation(self.get_number_of_periods()) for shift in self.shifts]
        np.savetxt(filename, np.array(shift_array), fmt='%s', delimiter=';')
    
    def write_solution_to_csv(self, instance, path = '.'):
        
        filename = path + '/' + instance.instance_name + '_sol.csv'
        
        self.write_to_csv(filename)
        


    
  

In [ ]:

shift_schedule = ShiftSchedule.from_file("test_data/sol1.csv")

shift_schedule.write_to_csv("test_data/sol2.csv")

## Rule-Related Data Structures and Functions



Below, we provide some data structures for providing a unified treatment of rule checking and reporting.

First, we have a namded tuple called `Penalty` which simply forms a tuple of a string with a rule name and a numerical value (a penalty).

Then, there is a class called `RuleEvaluationResult`. The attributes of this class are:
- a list of strings with hard rule violations
- a list of Penalty-objects.

This class can be used on different levels. We can
- explicitly add a hard rule violation or a penalty
- add another RuleEvaluationResult (this way, we can, for example compute the results for multiple shifts and combine them for a full shift schedule.

Then, there are various functions for computing total penalties and data frames that list details about the different penality types.

**TODO: We should create a small example rule set and a small schedule that we can use here as a dummy example**


In [ ]:
#export


Penalty=namedtuple('Penalty',['rule','penalty'])
                              


class RuleEvaluationResult:
    def __init__(self):                        
        self.hard_rule_violations = []
        self.penalties = []
        
    def add_hard_rule_violation(self, rule_violation):
        self.hard_rule_violations.append(rule_violation)
        
    def add_penalty(self, rule, penalty):
        self.penalties.append(Penalty(rule, penalty))
        
    def add_rule_evaluation_result(self, rule_evaluation_result):
        self.hard_rule_violations += rule_evaluation_result.hard_rule_violations
        self.penalties += rule_evaluation_result.penalties
        
    def get_df_hard_rule_violations(self):
        return pd.DataFrame(self.hard_rule_violations)
        
    def get_df_grouped_penalties(self):
        return pd.DataFrame(self.penalties).groupby(['rule'])['penalty'].sum().reset_index().style.hide_index()
    
    def get_df_detailed_penalties(self):
        return pd.DataFrame(self.penalties).style.hide_index()
        
    def get_total_penalty(self):
        return pd.DataFrame(self.penalties)['penalty'].sum()
    
        
    def is_feasible(self):
        return len(self.hard_rule_violations) == 0
        



The following function takes a schedule and a rule set and returns an EvaluationResults-Object.

Note that the rule_set is a class that needs to be implemented for a given problem type / instance-set. The rule_set needs to implement all the methods that are called below, namely:
- check_activity_block_rules
- check_work_block_rules
- check_shift_rules
- check_demand_coverage

At the moment, this package contains two implementations of such rule sets, for the Demassey-instances and for the Dahmen-instances.
    


In [ ]:
#export

def evaluate_shift_schedule(shift_schedule, rule_set):
    evaluation_results = RuleEvaluationResult()
    
    shift_id = 0
    for shift in shift_schedule.shifts:
        
        for activity_block in shift.activity_blocks:
            evaluation_results.add_rule_evaluation_result(rule_set.check_activity_block_rules(activity_block))
                                   
        for block in shift.work_and_break_blocks:
            if block.is_work():
                evaluation_results.add_rule_evaluation_result(rule_set.check_work_block_rules(block, shift_id))
        
        evaluation_results.add_rule_evaluation_result(rule_set.check_shift_rules(shift, shift_id))
    shift_id = 0
        
    for p in range(rule_set.instance.number_of_periods):
        for activity in range(rule_set.instance.number_of_activities):
            covered_demand = shift_schedule.get_demand_coverage(p, activity)
            res = rule_set.check_demand_coverage(p, activity, covered_demand)
            
            #if len(res.penalties) > 0:
            #    print (f"Period {p} Activity {activity} Penalty {res}")
            evaluation_results.add_rule_evaluation_result(res)
        
        
    return evaluation_results

Below two helper functions for computing covered demands.

They assume that the instance object has the two attributes `number_of_periods` and `number_of_activities`

In [ ]:
#export

def get_covered_activity_demands_per_period(shift_schedule, instance):
        
           
        covered_demands = []
        
        for activity in range(instance.number_of_activities):
            covered_demand = []
            for p in range(instance.number_of_periods):
                covered_demand.append(shift_schedule.get_demand_coverage(p, activity))
                
            covered_demands.append(covered_demand)
        
        return covered_demands
    
def get_covered_aggregate_demand_per_period(shift_schedule, instance):
        
    
        covered_demand = []
        
        for p in range(instance.number_of_periods):
            cover = 0
            for activity in range(instance.number_of_activities):       
                cover += shift_schedule.get_demand_coverage(p, activity)
                
            covered_demand.append(cover)
        
        return covered_demand

## Exporting

In [ ]:


#hide
from nbdev.export import *
notebook2script()



Converted 00_data_stuctures.ipynb.
Converted 00_reporting.ipynb.
Converted 01_demassey.ipynb.
Converted 02_dahmen.ipynb.
Converted index.ipynb.
